# ASSIGNED Changing Thiele Modulus

We consider the reaction $A \rightarrow B$ in a packed bed reactor that has a rate of $r = k C_A^2$ $k$=2.25 &times; 10<sup>5</sup> cm<sup>3</sup>/mol/s.
The gas feed consists of A and an inert, each at a molar flow of 10 mol/s. The total pressure is 4 atm at a temperature of 550K. The desired conversion is 75%.
The catalyst is a spherical pellet with a radius of 0.45 cm. The pellet density is &rho;<sub>p</sub> = 0.68 g/cm<sup>3</sup> and the bed density is &rho;<sub>b</sub> = 0.60 g / cm<sup>3</sup>. The effective diffusivity of A is 0.008 cm<sup>2</sup>/s.

-   Assume that the gas-phase concentration and the catalyst surface concentrations are the same and calculate the effectiveness factor at the entrance of the reactor and the exit of the reactor. Explain the trend you observe.

-   Now that you know the effective factor varies with the concentration of A, find the mass of catalyst required to achieve 75% conversion. Compare your results to the mass required if the catalyst had no mass transfer limitations.

# Solution

To compute the effectiveness factor, we have to use the Thiele modulus for power law kinetics. The Thiele modulus will depend on the gas phase concentration, so it will be different at the entrance and exit.

In [1]:
k = 2.25e5  # mol/cm^3/s
P = 4 # atm
T = 550 # K
R = 82.057 # cm^3 atm /mol/K

Fa0 = 10 # mol / s
Ca0 = 0.5 * P / R / T
v0 = Fa0 / Ca0
n = 2

radius = 0.45 # cm
De = 0.008 # cm^2/s

rhob = 0.60 # bed density g / cm^3
rhop = 0.68 # pellet density g/cm^3


Phi_0 = radius / 3 * ((n + 1) * k * Ca0**(n - 1) / (2 * De))**0.5
eta_0 = 1 / Phi_0 * ( 1 / np.tanh(3 * Phi_0) - 1 / (3 * Phi_0))
print('The effectiveness factor at the entrance is {}'.format(eta_0))

The effectiveness factor at the entrance is 0.14626030683471525

At the exit, we have converted 75% of A to products, so $Fa = F_{A0} (1 - 0.75)$

In [1]:
X = 0.75
Fa_exit = Fa0 * (1 - X)
Ca_exit = Fa_exit / v0
Phi_exit = radius / 3 * ((n + 1) * k * Ca_exit**(n - 1) / (2 * De))**0.5
eta_exit = 1 / Phi_exit * ( 1 / np.tanh(3 * Phi_exit) - 1 / (3 * Phi_exit))
print('The effectiveness factor at the exit is {}'.format(eta_exit))

The effectiveness factor at the exit is 0.27667202106955185

It appears that the catalyst becomes more effective at lower concentration of A. That happens because Phi gets smaller, and so the effectiveness factor increases.

For the second part, we have to integrate the mole balance. The effectiveness factor will change with conversion, and so we have to compute it at each step. I will use quad here since we can solve for the volume directly that way:

$V = \int_{F_{A0}}^{F_A} dF_A/r_A$

The volume we get is the pellet volume, and to convert it to reactor volume we multiply by the pellet density to get the pellet mass, and then divide by the bed density to get the bed volume.

In [1]:
from scipy.integrate import quad

def integrand(Fa):
    Ca = Fa / v0
    Phi = radius / 3 * ((n + 1) * k * Ca**(n - 1) / (2 * De))**0.5
    eta = 1 / Phi * ( 1 / np.tanh(3 * Phi) - 1 / (3 * Phi))
    r = eta * k * Ca**2
    ra = -r
    return 1 / ra

pellet_vol, _ = quad(integrand, Fa0, Fa0 * (1 - X))

pellet_mass = pellet_vol * rhop
reactor_volume = pellet_mass / rhob
print('The reactor volume is {:1.3g} cm^3'.format(reactor_volume))

The reactor volume is 3.61e+05 cm^3

In the absence of diffusion limits, the effectiveness factor is one.

In [1]:
from scipy.integrate import quad

def integrand(Fa):
    Ca = Fa / v0
    eta = 1
    r = eta * k * Ca**2
    ra = -r
    return 1 / ra

pellet_vol, _ = quad(integrand, Fa0, Fa0 * (1 - X))

pellet_mass = pellet_vol * rhop
reactor_volume = pellet_mass / rhob
print('The reactor volume is {:1.3g} cm^3'.format(reactor_volume))

The reactor volume is 7.69e+04 cm^3

The volume required for no diffusion limit is smaller because all of the pellet volume is effective in the reactions.